<img src="images/img.png" />

# CS5228 Project, Group 32

In [12]:
# Auto reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
from tqdm import tqdm

In [14]:
from util.DataMining import split_dataframe, split_dataframe_flex
from util.DataMining import (
    RandomForestMining,
    RandomForestMiningByModel,
    GradientBoostingMining,
    GradientBoostingMiningByModel,
    XGBoostMining,
    XGBoostMiningByMake,
    LinearRegressionMining,
    LinearRegressionMiningByModel,
)

## Data mining

### Load the preprocessed training and test data

In [41]:
train_file = './data/train_preprocessed_impute.csv'
test_file = './data/test_preprocessed.csv'

df = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

### Train XGB on the preselected 18 attributes

In [49]:
selected_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'engine_cap',
    'no_of_owners',
    'depreciation',
    'power',
    'road_tax',
    'mileage',
    'omv_arf_ratio',
    'dereg_coe_ratio',
    'car_age',
    'omv',
    'arf',
    'price',
    'coe car',
    'parf car'
]

df_filtered = df[selected_columns]

run_times, rmse_sum = 1, 0
for i in tqdm(range(run_times), desc='Running XGBoost'):
    train_drop_cols = ['price']
    x_train, x_test, y_train, y_test = split_dataframe(df_filtered, train_drop_cols)
    rmse_sum += RandomForestMining(x_train, x_test, y_train, y_test)
print('Average RMSE:', round(rmse_sum / run_times))

Running XGBoost:   0%|                                                                                                                                                     | 0/1 [00:00<?, ?it/s]/Users/enqurance/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/enqurance/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/enqurance/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Cross-validated RMSE scores: [23050.56632623 21247.0158987  19524.58627001 25491.57628489
 27168.68061599]
Mean CV RMSE: 23296.485079163635


Running XGBoost: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [12:22<00:00, 742.24s/it]

Running not in develop mode
RMSE on test data: 26755.482461221243
Average RMSE: 26755


In [34]:
x_train.head()

,make,model,type_of_vehicle,engine_cap,no_of_owners,depreciation,power,road_tax,mileage,omv_arf_ratio,dereg_coe_ratio,car_age,omv,arf,coe car,parf car
14809,51.0,157.0,2.0,1595.0,2.0,19460.0,90.0,740.0,85000.000000,1.005872,0.746056,7.0,32032.0,31845.0,0.0,1.0
13329,58.0,402.0,6.0,3799.0,6.0,45870.0,357.0,5454.0,98000.000000,1.000000,0.535894,15.0,106010.0,106010.0,1.0,0.0
11865,8.0,26.0,6.0,1499.0,5.0,21790.0,100.0,684.0,113000.000000,0.856765,0.982751,6.0,34358.0,40102.0,0.0,1.0
14957,8.0,418.0,1.0,647.0,2.0,21090.0,125.0,948.0,82604.147002,1.746407,0.485814,8.0,45927.0,26298.0,0.0,1.0
23423,29.0,413.0,8.0,1496.0,1.0,16630.0,96.0,682.0,32000.000000,0.992359,1.091874,5.0,20391.0,20548.0,0.0,1.0


### Predict on Kaggle test data and save the result file

In [47]:
df_selected = df[selected_columns]

x_train, y_train = df_selected.drop(columns=['price']), df[['price']]
x_test = df_test[x_train.columns]

res_nomodel = RandomForestMining(x_train, x_test, y_train, dev=True)
res_nomodel.reset_index(inplace=True)
res_nomodel.rename(columns={'index': 'Id'}, inplace=True)
res_nomodel.to_csv('./data/rf_result.csv', index=False)

/Users/enqurance/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


### Train XGB for each value of `make`

In [212]:
run_times, rmse_sum = 5, 0
for i in tqdm(range(run_times), desc='Running XGBoost'):
    train_drop_cols = ['price']
    test_cols = ['price', 'make']
    x_train, x_test, y_train, y_test = split_dataframe_flex(df, train_drop_cols, test_cols)
    rmse_sum += XGBoostMiningByMake(x_train, x_test, y_train, y_test)
print('Average RMSE:', round(rmse_sum / run_times))

Running XGBoost:   0%|                                                                                                                                                     | 0/5 [00:00<?, ?it/s]/Users/enqurance/Desktop/NUS/Sem1_AY2024-2025/CS5228/Projects/CS5228_Project/util/DataMining.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_models = pd.concat([missing_models, row_df])
Running XGBoost:  20%|████████████████████████████                                                                                                                | 1/5 [02:23<09:34, 143.58s/it]

Running not in develop mode
RMSE on test data: 86576.02580016718


/Users/enqurance/Desktop/NUS/Sem1_AY2024-2025/CS5228/Projects/CS5228_Project/util/DataMining.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_models = pd.concat([missing_models, row_df])
Running XGBoost:  40%|████████████████████████████████████████████████████████                                                                                    | 2/5 [04:49<07:14, 144.74s/it]

Running not in develop mode
RMSE on test data: 97104.87752028811


/Users/enqurance/Desktop/NUS/Sem1_AY2024-2025/CS5228/Projects/CS5228_Project/util/DataMining.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_models = pd.concat([missing_models, row_df])
Running XGBoost:  60%|████████████████████████████████████████████████████████████████████████████████████                                                        | 3/5 [07:13<04:49, 144.72s/it]

Running not in develop mode
RMSE on test data: 93299.2585681242


/Users/enqurance/Desktop/NUS/Sem1_AY2024-2025/CS5228/Projects/CS5228_Project/util/DataMining.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_models = pd.concat([missing_models, row_df])
Running XGBoost:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████                            | 4/5 [09:33<02:22, 142.90s/it]

Running not in develop mode
RMSE on test data: 92199.50612841758


/Users/enqurance/Desktop/NUS/Sem1_AY2024-2025/CS5228/Projects/CS5228_Project/util/DataMining.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  missing_models = pd.concat([missing_models, row_df])
Running XGBoost: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [12:00<00:00, 144.16s/it]

Running not in develop mode
RMSE on test data: 91798.99605347653
Average RMSE: 92196


## Predict on the Kaggle test data and save the result file

In [ ]:
x_train, y_train = df.drop(columns=['price']), df[['price', 'make']]
x_test = df_test[x_train.columns]
res_model =  XGBoostMiningByMake(x_train, x_test, y_train, dev=True)

In [ ]:
res_model.to_csv('./data/xgb_by_model_result.csv', index=False)